# Audio classification on the Urban SoundsII dataset with CLAP

###  Goal of the notebook
In this notebook you can do audio classification with CLAP.

### CLAP
CLAP (Contrastive Language-Audio Pretraining) is a neural network trained on a variety of (audio, text) pairs. It can be instructed in to predict the most relevant text snippet, given an audio, without directly optimizing for the task.

Source: https://huggingface.co/laion/larger_clap_general
CLAP paper: https://arxiv.org/abs/2211.06687

In this notebook we will use two CLAP models:
1. larger_clap_music_and_speech
2. larger_clap_general

### Using 🤗 datasets and 🤗transformers
The dataset is hosted on the Huggingface Hub at: https://huggingface.co/datasets/MichielBontenbal/UrbanSoundsII

(This is a new version of the same dataset as the old dataset got corrupted.)

This dataset contains nine classes of audio events in an urban environment. 

In this notebook we will use 🤗  ```dataset``` library to load this dataset. 

And we'll use the 🤗 ```transformers``` library to run the CLAP model. Please find more info: https://huggingface.co/docs/transformers/model_doc/clap 


### Contents
0. Install packages & check versions
1. Inspection of dataset
2. CLAP on the Urban Sounds Amsterdam dataset


## 0. Install packages

In [5]:
#!pip install datasets

In [6]:
#!pip install soundfile

In [7]:
#!pip install librosa

In [8]:
#%pip install datasets\[audio\]

In [9]:
pip install numpy==1.26

Note: you may need to restart the kernel to use updated packages.


In [10]:
#check python version
import platform
print(platform.python_version())

3.12.7


In [11]:
import numpy as np
print(f'numpy={np.__version__}')
import soundfile
print(f'soundfile={soundfile.__version__}')
import librosa
print(f'librosa={librosa.__version__}')
import IPython
print(f'ipython={IPython.__version__}')


numpy=1.26.0
soundfile=0.12.1
librosa=0.10.2.post1
ipython=8.27.0


In [12]:
import datasets
print(f'datasets={datasets.__version__}')
import transformers
print(f'transformers={transformers.__version__}')

datasets=3.1.0
transformers=4.46.3


## 1. Inspection of the dataset

In [13]:
from datasets import load_dataset
#
dataset = load_dataset("MichielBontenbal/UrbanSoundsII")


Resolving data files:   0%|          | 0/223 [00:00<?, ?it/s]

In [14]:
#The ESC50 dataset is one of the very few other datasets on Environmental Sound classification
#You could try this as an alternative
#dataset = load_dataset("confit/esc50-demo", "fold1")

In [15]:
#inspect the dataset
#dataset = ds
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'label'],
        num_rows: 223
    })
})

In [16]:
#Inspect one sample from 
example = dataset['train']['audio'][0]
label = dataset['train']['label'][0]

You may notice that the audio column contains several features. Here’s what they are:

- path: the path to the downloaded (and converted) audio file
- array: The decoded audio data, represented as a 1-dimensional NumPy array.
- sampling_rate. The sampling rate of the audio file.

In [17]:
example

{'path': '/Users/michielbontenbal/.cache/huggingface/datasets/downloads/c6de4f4db3f3eb3f95189e05acc6bbd4db6e00537183441353e0477e89b96e45',
 'array': array([-0.00015259, -0.00012207, -0.00021362, ...,  0.00015259,
         0.00018311,  0.        ]),
 'sampling_rate': 44100}

In [18]:
#print the label data
print(dataset['train']['label'])
print(len(dataset['train']['label']))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]
223


In [19]:
#inspecting the audio array
array = dataset["train"]["audio"][0]["array"]
sampling_rate = example["sampling_rate"]
print(array.shape)
print(array)
print(type(array))
print(sampling_rate)

(441000,)
[-0.00015259 -0.00012207 -0.00021362 ...  0.00015259  0.00018311
  0.        ]
<class 'numpy.ndarray'>
44100


## 2. CLAP on the Urban Sounds Amsterdam dataset


In [20]:
#create a random number to select from dataset
import random

random_number = random.randint(0, len(dataset['train']['audio']))
random_number

71

### Runnning it with "Larger CLAP music and speech" model

In [21]:
#Script to load a random number out of the dataset
from transformers import ClapModel, ClapProcessor
from datasets import load_dataset
from transformers import pipeline
import IPython

example=dataset['train']['audio'][random_number]
audio = dataset["train"]["audio"][random_number]["array"]

audio_classifier = pipeline(task="zero-shot-audio-classification", model="laion/larger_clap_music_and_speech")
output = audio_classifier(audio, candidate_labels=["Motorcycle", "Moped", 'Claxon','Alarm', 'Silence','Loud people','Talking','Gunshot', 'Slamming door','Music'])
print(output[0],'\n',output[1])
print(random_number)
IPython.display.Audio(example["array"], rate=example['sampling_rate'])

{'score': 0.6245052814483643, 'label': 'Motorcycle'} 
 {'score': 0.3166671097278595, 'label': 'Moped'}
71


### Runnning it with "Larger CLAP general" model

In [22]:
# This is code to convert the given labels (0,1,2,3,4,5,6,7,8,9) to a real string. 
# create a dictionary the converts the class folders to real names
label_dict ={0:'Gunshot', 1:'Moped alarm', 2:'Moped', 3:'Claxon', 4:'Slamming door', 5:'Screaming', 6:'Motorcycle', 7:'Talking', 8:'Music'}
print('The given labels are: ')
for i in range(0,9):
    print(label_dict[i])

The given labels are: 
Gunshot
Moped alarm
Moped
Claxon
Slamming door
Screaming
Motorcycle
Talking
Music


In [23]:
#larger_clap_general
from transformers import ClapModel, ClapProcessor
from transformers import pipeline
from datasets import load_dataset
import IPython

example=dataset['train']['audio'][random_number]
audio = dataset["train"]["audio"][random_number]['array']

audio_classifier = pipeline(task="zero-shot-audio-classification", model="laion/larger_clap_general")
output = audio_classifier(audio, candidate_labels=["Gunshot", "Moped", 'Moped alarm','Claxon','Screaming', 'Motorcycle','Talking', 'Slamming door','Music', 'Silence'])

predicted_label = output[0]['label']
print(f'Prediction: {predicted_label}')

label_name =label_dict[dataset['train']['label'][i]]
print(f'The given label: {label_name}')

if label_name == output[0]['label']:
    print("This is correct")
else:
    print('This is false')
print(f'Probability: {round(output[0]["score"],3)}')

IPython.display.Audio(example['array'], rate=example['sampling_rate'])

Prediction: Moped
The given label: Gunshot
This is false
Probability: 0.648
